In [ ]:
  #connexion a gee
import ee
import geemap
ee.Authenticate()
ee.Initialize(project = 'projetapplicatiremotesening')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pycrs

In [ ]:
shapefile_path = '/content/drive/MyDrive/projet_dakhla /zone0.shp'
shapefile_path1 = '/content/drive/MyDrive/projet_dakhla /zone1.shp'
shapefile_path2 = '/content/drive/MyDrive/projet_dakhla /zone2.shp'
roi = geemap.shp_to_ee(shapefile_path)
roi1 = geemap.shp_to_ee(shapefile_path1)
roi2 = geemap.shp_to_ee(shapefile_path2)
Map = geemap.Map()

# Add the layers to the map
Map.addLayer(roi)
Map.addLayer(roi1)
Map.addLayer(roi2)

# Center the map on the ROI and display
Map.centerObject(roi, 10)
Map


Map(center=[22.105702204625008, -16.083888888888858], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
print(roi)

ee.FeatureCollection({
  "functionInvocationValue": {
    "functionName": "Collection",
    "arguments": {
      "features": {
        "arrayValue": {
          "values": [
            {
              "functionInvocationValue": {
                "functionName": "Feature",
                "arguments": {
                  "geometry": {
                    "functionInvocationValue": {
                      "functionName": "GeometryConstructors.Polygon",
                      "arguments": {
                        "coordinates": {
                          "constantValue": [
                            [
                              [
                                -15.820277777777777,
                                22.26861111111111
                              ],
                              [
                                -15.820277777777777,
                                21.942777777777778
                              ],
                              [
                       

In [ ]:
import ee
import geemap
ee.Initialize()



# Function to mask clouds
def mask_clouds(image):
    cloud_mask = image.select('QA60').lt(1)
    return image.updateMask(cloud_mask)

# Load Sentinel-2 Surface Reflectance data
sentinel2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
             .filterBounds(roi)
             .filterDate('2020-01-01', '2023-12-31')
             .map(mask_clouds)
             .median()
             .clip(roi))

# Calculate the Iron Oxide Index (Red / Blue)
iron_oxide = sentinel2.normalizedDifference(['B4', 'B2']).rename('Iron_Oxide')

# Calculate the Clay Minerals Index (SWIR / NIR)
clay_minerals = sentinel2.normalizedDifference(['B11', 'B8']).rename('Clay_Minerals')

# Set visualization parameters for Iron Oxide
iron_oxide_vis = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'red']
}

# Set visualization parameters for Clay Minerals
clay_minerals_vis = {
    'min': -1,
    'max': 1,
    'palette': ['white', 'green']
}
# Add the layers to the map
Map.addLayer(iron_oxide, iron_oxide_vis, 'Iron Oxide')
Map.addLayer(clay_minerals, clay_minerals_vis, 'Clay Minerals')
Map


Map(bottom=114799.0, center=[22.183590128241153, -15.816073596065383], controls=(WidgetControl(options=['posit…

In [ ]:
iron_oxide_stats = iron_oxide.reduceRegion(
    reducer=ee.Reducer.mean().combine(
        reducer2=ee.Reducer.minMax(),
        sharedInputs=True
    ).combine(
        reducer2=ee.Reducer.stdDev(),
        sharedInputs=True
    ),
    geometry=roi,
    scale=10,
    maxPixels=1e13
)

# Compute statistics for Clay Minerals
clay_minerals_stats = clay_minerals.reduceRegion(
    reducer=ee.Reducer.mean().combine(
        reducer2=ee.Reducer.minMax(),
        sharedInputs=True
    ).combine(
        reducer2=ee.Reducer.stdDev(),
        sharedInputs=True
    ),
    geometry=roi,
    scale=10,
    maxPixels=1e13
)

# Print the results
print('Iron Oxide Statistics:', iron_oxide_stats.getInfo())
print('Clay Minerals Statistics:', clay_minerals_stats.getInfo())

In [ ]:
#adjusting the calculations for the multiplication process
mineral_zones = iron_oxide.multiply(clay_minerals).rename('Mineral_Zones')
mineral_zones_vis = {
    'min': 0.009,
    'max': 0.17,
     'palette': ['blue', 'lightblue', 'yellow', 'orange', 'red']
}
Map = geemap.Map()
# Add the mineral zones layer to the map
Map.addLayer(mineral_zones, mineral_zones_vis, 'Mineral Zones')
# Center the map on the ROI
Map.centerObject(roi, 10)
# Display the map
Map

Map(center=[22.105702204625008, -16.083888888888858], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
# do the same for zone 2 and zone 3 , then export the image to my drive
import ee
import geemap
ee.Initialize()
# Function to mask clouds
def mask_clouds(image):
    cloud_mask = image.select('QA60').lt(1)
    return image.updateMask(cloud_mask)

# Load Sentinel-2 Surface Reflectance data
sentinel2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
             .filterBounds(roi2)
             .filterDate('2020-01-01', '2023-12-31')
             .map(mask_clouds)
             .median()
             .clip(roi2))
# Calculate the Iron Oxide Index (Red / Blue)
iron_oxide = sentinel2.normalizedDifference(['B4', 'B2']).rename('Iron_Oxide')
# Calculate the Clay Minerals Index (SWIR / NIR)
clay_minerals = sentinel2.normalizedDifference(['B11', 'B8']).rename('Clay_Minerals')
# Set visualization parameters for Iron Oxide
iron_oxide_vis = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'red']
}
# Set visualization parameters for Clay Minerals
clay_minerals_vis = {
    'min': -1,
    'max': 1,
    'palette': ['white', 'green']
}
#adjusting the calculations for the multiplication process
mineral_zones = iron_oxide.multiply(clay_minerals).rename('Mineral_Zones')
mineral_zones_vis = {
    'min': 0.009,
    'max': 0.17,
     'palette': ['blue', 'lightblue', 'yellow', 'orange', 'red']
}
Map = geemap.Map()
# Add the mineral zones layer to the map
Map.addLayer(mineral_zones, mineral_zones_vis, 'Mineral Zones')
# Center the map on the roi2
Map.centerObject(roi2, 10)
# Display the map
Map

Map(center=[22.498195683815567, -15.204861111110658], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
# do the same for zone 2 and zone 3 , then export the image to my drive
import ee
import geemap
ee.Initialize()

# Function to mask clouds
def mask_clouds(image):
    cloud_mask = image.select('QA60').lt(1)
    return image.updateMask(cloud_mask)

# Load Sentinel-2 Surface Reflectance data
sentinel2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
             .filterBounds(roi1)
             .filterDate('2020-01-01', '2023-12-31')
             .map(mask_clouds)
             .median()
             .clip(roi1))

# Calculate the Iron Oxide Index (Red / Blue)
iron_oxide = sentinel2.normalizedDifference(['B4', 'B2']).rename('Iron_Oxide')

# Calculate the Clay Minerals Index (SWIR / NIR)
clay_minerals = sentinel2.normalizedDifference(['B11', 'B8']).rename('Clay_Minerals')

# Set visualization parameters for Iron Oxide
iron_oxide_vis = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'red']
}

# Set visualization parameters for Clay Minerals
clay_minerals_vis = {
    'min': -1,
    'max': 1,
    'palette': ['white', 'green']
}
#adjusting the calculations for the multiplication process
mineral_zones = iron_oxide.multiply(clay_minerals).rename('Mineral_Zones')
mineral_zones_vis = {
    'min': 0.009,
    'max': 0.17,
     'palette': ['blue', 'lightblue', 'yellow', 'orange', 'red']
}
Map = geemap.Map()
# Add the mineral zones layer to the map
Map.addLayer(mineral_zones, mineral_zones_vis, 'Mineral Zones')
# Center the map on the roi1
Map.centerObject(roi1, 10)
# Display the map
Map

Map(center=[22.593042222967043, -15.78430555555584], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
# add all images in one zone


In [ ]:
mineral_zones_stats = mineral_zones.reduceRegion(
    reducer=ee.Reducer.mean().combine(
        reducer2=ee.Reducer.minMax(),
        sharedInputs=True
    ).combine(
        reducer2=ee.Reducer.stdDev(),
        sharedInputs=True
    ),
    geometry=roi,
    scale=10,
    maxPixels=1e13
)
print(mineral_zones_stats)

ee.Dictionary({
  "functionInvocationValue": {
    "functionName": "Image.reduceRegion",
    "arguments": {
      "geometry": {
        "functionInvocationValue": {
          "functionName": "Collection.geometry",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection",
                "arguments": {
                  "features": {
                    "arrayValue": {
                      "values": [
                        {
                          "functionInvocationValue": {
                            "functionName": "Feature",
                            "arguments": {
                              "geometry": {
                                "functionInvocationValue": {
                                  "functionName": "GeometryConstructors.Polygon",
                                  "arguments": {
                                    "coordinates": {
                                      "cons